<a href="https://colab.research.google.com/github/milorico/KDD2020-01/blob/master/KDD_Project_UNAB_2020_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aplicacion de la metodología CRISP-DM para descubrir conocimiento  en la información de Airbnb en anuncios de hospedaje para Amsterdam y Berlin 2020 - 2021**

# Introducción

CRISP-DM, proporciona una representación para la mineria de datos como se hace en la industria del software con los modelos de ciclo vital en el desarrollo de software. El modelo CRISP-DM cubre las fases de un proyecto, como un modelo de procesos, es un metodo probado para orientar proyectos de este tipo, en donde se hara mineria de datos. 

Compararemos las tendencias de Airbnb entre Amsterdam y Berlin durante un periodo de extension de un año, abril 2020 - 2021. Examinaremos las diferencias entre los listados y las tendencias anuales entre las dos ciudades europeas.



# Comprensión del negocio

Los tipos de datos disponibles para la realizacion del proyecto son de origen de InsideAirbnb, una iniciativa de Airbnb para permitirle a todo publico acceder y descargar informacion sobre las ofertas y reseñas en su sistema clasificadas para diferentes ciudades al rededor del mundo. Especificamente para este proyecto, utilizaremos dos archivos para cada ciudad, Listing.csv y Calendar.csv, los cuales contienen informacion sobre precios, disponiblidad, tamaño, ubicacion, anfitrion y otros datos sobre cada oferta. 

Para el desarrollo de este proyecto, utilizaremos herramientas online como google Collab, las cuales nos facilitan el montaje y ejecucion de los modelos, asi mismo, la creacion de un cuaderno para entablar una conversacion con el lector mientras se aprecian las graficas, ejecuta los codigos y se sacan conclusiones.

El negocio del hospedaje para turismo o celebraciones se ha modernizado notablemente rapido a comparacion de otro tipo de servicios. Gracias a esto, herramientas como Airbnb, Trivago, googleFlights, le permiten al arrendatario apreciar diferentes opciones con sus especificaciones e imagenes ilustrativas, acomodaciones y demas, pero como saber que tipo de hospedaje es el mas economico o rentable para diferentes presupuestos es el proposito de esta investigacion, queremos saber que ciudad de las dos escogidas ofrece acomodaciones a mejores precios considerando una media baja y otra alta ya que las acomodaciones que ofrece la herramienta son altamente variadas y enfocadas a dos publicos objetivos diferentes previamente mencionados, es por eso que con la ayuda de calculos estadisticos y herramientas graficadoras analizaremos las ofertas de cada ciudad para ofrecer respuestas a las preguntas de investigacion planteadas. 

